In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping , ModelCheckpoint
import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

In [2]:
base_model=tf.keras.applications.VGG16(weights='imagenet',input_shape=(224,224,3),include_top=False)

for layer in base_model.layers:
    layer.trainable=False

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


In [3]:
model=Sequential([
    base_model,
    Flatten(),
    Dense(128,activation='relu'), #Optional
    Dense(5,activation='softmax') #Mandatory
])

In [4]:
model.compile(optimizer=Adam(learning_rate=0.001),loss='categorical_crossentropy',metrics=['accuracy'])

In [5]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)              │ (None, 7, 7, 512)      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     3,211,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 5)              │           645 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 17,926,725 (68.39 MB)

 Trainable params: 3,212,037 (12.25 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [6]:
dataset,info=tfds.load('tf_flowers',with_info=True,as_supervised=True)

In [7]:
info

tfds.core.DatasetInfo(
    name='tf_flowers',
    full_name='tf_flowers/3.0.1',
    description="""
    A large set of images of flowers
    """,
    homepage='https://www.tensorflow.org/tutorials/load_data/images',
    data_dir='/root/tensorflow_datasets/tf_flowers/3.0.1',
    file_format=tfrecord,
    download_size=218.21 MiB,
    dataset_size=221.83 MiB,
    features=FeaturesDict({
        'image': Image(shape=(None, None, 3), dtype=uint8),
        'label': ClassLabel(shape=(), dtype=int64, num_classes=5),
    }),
    supervised_keys=('image', 'label'),
    disable_shuffling=False,
    nondeterministic_order=False,
    splits={
        'train': <SplitInfo num_examples=3670, num_shards=2>,
    },
    citation="""@ONLINE {tfflowers,
    author = "The TensorFlow Team",
    title = "Flowers",
    month = "jan",
    year = "2019",
    url = "http://download.tensorflow.org/example_images/flower_photos.tgz" }""",
)

In [7]:
n_classes=info.features['label'].num_classes
print(n_classes)

5


In [8]:
IMG_SIZE=224

In [9]:
images,labels=[],[]
for image,label in tfds.as_numpy(dataset['train']):
    image=tf.image.resize(image,(IMG_SIZE,IMG_SIZE))
    images.append(image)
    labels.append(label)

In [10]:
images=np.array(images)
labels=np.array(labels)

In [11]:
images.shape

(3670, 224, 224, 3)

In [12]:
labels.shape

(3670,)

In [13]:
from sklearn.model_selection import train_test_split

In [14]:
X_train,X_test,y_train,y_test=train_test_split(images,labels,test_size=0.2,random_state=42)

In [21]:
train_datagen=ImageDataGenerator(rescale=1./255)
test_datagen=ImageDataGenerator(rescale=1./255)


In [22]:
train_datagen=train_datagen.flow(
    tf.image.resize(X_train,(IMG_SIZE,IMG_SIZE)).numpy(), #Best Practice
    tf.keras.utils.to_categorical(y_train,num_classes=n_classes)
)

In [23]:
test_datagen=test_datagen.flow(
    tf.image.resize(X_test,(IMG_SIZE,IMG_SIZE)).numpy(),
    tf.keras.utils.to_categorical(y_test,num_classes=n_classes)
)

In [19]:
history=model.fit(train_datagen,validation_data=test_datagen,epochs=5)

Epoch 1/5


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


92/92 ━━━━━━━━━━━━━━━━━━━━ 72s 582ms/step - accuracy: 0.2857 - loss: 1.5970 - val_accuracy: 0.3202 - val_loss: 1.5582
Epoch 2/5
92/92 ━━━━━━━━━━━━━━━━━━━━ 22s 241ms/step - accuracy: 0.3401 - loss: 1.5349 - val_accuracy: 0.3433 - val_loss: 1.4770
Epoch 3/5
92/92 ━━━━━━━━━━━━━━━━━━━━ 23s 247ms/step - accuracy: 0.3806 - loss: 1.4491 - val_accuracy: 0.3678 - val_loss: 1.4078
Epoch 4/5
92/92 ━━━━━━━━━━━━━━━━━━━━ 23s 246ms/step - accuracy: 0.3943 - loss: 1.4024 - val_accuracy: 0.3910 - val_loss: 1.3666
Epoch 5/5
92/92 ━━━━━━━━━━━━━━━━━━━━ 22s 244ms/step - accuracy: 0.4265 - loss: 1.3553 - val_accuracy: 0.4564 - val_loss: 1.3413


In [20]:
base_model.trainable=True

In [17]:
model_fine=Sequential([
    base_model,
    Flatten(),
    Dense(128,activation='relu'),
    Dense(5,activation='softmax')
])

In [18]:
model_fine.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [24]:
history_fine=model_fine.fit(train_datagen,validation_data=test_datagen,epochs=5)

Epoch 1/5


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


KeyboardInterrupt: 

In [19]:
model_fine.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)              │ (None, 7, 7, 512)      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │     3,211,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 5)              │           645 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 17,926,725 (68.39 MB)

 Trainable params: 17,926,725 (68.39 MB)

 Non-trainable params: 0 (0.00 B)

In [16]:

for layer in base_model.layers:
    layer.trainable=True

**Hyper Parameter Tuning**

In [26]:
!pip install keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 14.1 MB/s eta 0:00:00


In [27]:
from kerastuner.tuners import RandomSearch
from tensorflow.keras.datasets import mnist

/tmp/ipython-input-211307991.py:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


In [28]:
(X_train,y_train), (X_test,y_test)=mnist.load_data()

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


In [33]:
# X_train=np.array(X_train)
# y_train=np.array(y_train)

In [34]:
# train_datagen=ImageDataGenerator(rescale=1./255)
# test_datagen=ImageDataGenerator(rescale=1./255)


In [37]:
X_train=X_train.astype(float)/255
X_test=X_test.astype(float)/255

In [38]:
y_train=tf.keras.utils.to_categorical(y_train,num_classes=10)
y_test=tf.keras.utils.to_categorical(y_test,num_classes=10)

In [36]:
# train_datagen=train_datagen.flow(
#     tf.image.resize(X_train,(28,28)).numpy(), #Best Practice
#     tf.keras.utils.to_categorical(y_train,num_classes=10)
# )

ValueError: Input data in `NumpyArrayIterator` should have rank 4. You passed an array with shape (60000, 10)

In [ ]:
# test_datagen=test_datagen.flow(
#     tf.image.resize(X_test,(28,28)).numpy(),
#     tf.keras.utils.to_categorical(y_test,num_classes=10)
# )

In [44]:
def build_model(hp):
  model=Sequential()
  model.add(Flatten(input_shape=(28,28))) #Flatten Input Images
  model.add(
      Dense(units=hp.Int('units_1',min_value=32,max_value=256,step=32),activation='relu')
  )
  model.add(
      Dense(units=hp.Int('units_2',min_value=32,max_value=256,step=32),activation='relu')
  )

  model.add(Dense(10,activation='softmax'))

  model.compile(optimizer=Adam(learning_rate=hp.Choice('lr',[0.001,0.01,0.1])),loss='categorical_crossentropy',metrics=['accuracy'])

  return model

In [45]:
tuner=RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    directory='simple_tuning',
    project_name='mnist_tuning'
)

/usr/local/lib/python3.12/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [46]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test),batch_size=32)

Trial 5 Complete [00h 00m 31s]
val_accuracy: 0.9545000195503235

Best val_accuracy So Far: 0.9739000201225281
Total elapsed time: 00h 02m 50s


In [50]:
best_model=tuner.get_best_models(num_models=1)[0]

In [51]:
best_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten (Flatten)               │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       100,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         4,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 10)             │           330 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 104,938 (409.91 KB)

 Trainable params: 104,938 (409.91 KB)

 Non-trainable params: 0 (0.00 B)

In [58]:
best_params=tuner.get_best_hyperparameters()[0]

In [59]:
best_params.values

{'units_1': 128, 'units_2': 32, 'lr': 0.001}

In [70]:
model_2=Sequential([
    Flatten(input_shape=(28,28)),
    Dense(128,activation='relu'),
    Dense(32,activation='relu'),
    Dense(10,activation='softmax')

])

/usr/local/lib/python3.12/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [71]:
model_2.compile(optimizer=Adam(learning_rate=0.001),loss='categorical_crossentropy',metrics=['accuracy'])

In [72]:
early_stopping=EarlyStopping(monitor='val_loss',patience=3,restore_best_weights=True)

In [73]:
model_checkpoint=ModelCheckpoint('best_model.keras',monitor='val_loss',save_best_only=True)

In [74]:
model_2.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=50,callbacks=[early_stopping,model_checkpoint])

Epoch 1/50
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - accuracy: 0.8654 - loss: 0.4577 - val_accuracy: 0.9552 - val_loss: 0.1492
Epoch 2/50
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9663 - loss: 0.1116 - val_accuracy: 0.9705 - val_loss: 0.1000
Epoch 3/50
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9759 - loss: 0.0763 - val_accuracy: 0.9732 - val_loss: 0.0860
Epoch 4/50
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9816 - loss: 0.0580 - val_accuracy: 0.9738 - val_loss: 0.0860
Epoch 5/50
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9868 - loss: 0.0417 - val_accuracy: 0.9759 - val_loss: 0.0791
Epoch 6/50
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9895 - loss: 0.0318 - val_accuracy: 0.9795 - val_loss: 0.0736
Epoch 7/50
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9911 - loss: 0.0290 - val_accuracy: 0.9736 - val_loss: 0.0984
Epoch 8/50
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9927 - loss: 0.0242 - 